In [1]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")

import numpy as np
import math
import pandas as pd
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal

import data_analysis_utils as utils
import calibrate_power as cbp
from read_scope_data import read_trc_data
from crds_tools import read_excel_sheets, get_power_solenoid

%matplotlib qt

In [15]:
ifn = r"C:\data\epfl\diagnostic source\CRDS\solenoid\20230601_15mTorr_0.15lnmin.xlsx"
dic_15mT = read_excel_sheets(ifn)

['330W-20W', '400W-20W', '510W-10W', '590W-10W']


In [2]:
start = 115000
stop = 125000
pressure = 15

pwr_ls = get_power_solenoid(pressure, start, stop)

Reading data...
Done
Power = 307.1471836497802 W
Reading data...
Done
Power = 378.32275259060384 W
Reading data...
Done
Power = 495.4730699544068 W
Reading data...
Done
Power = 574.3750972509141 W


In [4]:
print(pwr_ls[0])

300.0


In [18]:
def get_real_power(path, pressure):

    file_ls = utils.get_files_in_folder(path)
    for file in file_ls:
        if f"{pressure}mT" in file:
            
    
    ifn_light = path + f"C3-{power}W-{pressure}mT-00000.trc"
    light, tarr = read_trc_data(ifn_light)

    ifn_Vtrig = path + f"C4-{power}W-{pressure}mT-00000.trc"
    Vtrig, tarr = read_trc_data(ifn_Vtrig)

    fig, ax = plt.subplots(nrows=2, sharex=True)

    ax[0].plot(tarr*1e3, light)
    ax[0].set_ylabel('Light')

    ax[1].plot(tarr*1e3, Vtrig)
    ax[1].set_ylabel('TTL')
    ax[1].set_xlabel('time (ms)')

    start = 20000
    stop = 24000
    ax[0].plot(tarr[start:stop]*1e3, light[start:stop]) # plot range to take average
    lavg = np.average(light[start:stop]) * 1e3 # take average light signal conver V to mV
    pwr = cbp.ltop_solenoid(lavg, pressure) # Calibrated power according to CW light signal
    print(f'Power = {pwr} W')

    return pwr


pressure = 15
power = 300
real_power = get_real_power(path, pressure, 0.15)

In [ ]:

def save_to_npy(filepath, columns, skip_rows=0):
    # Check if the .npy file already exists
    npy_file_path = f"{sheet_name}.npy"
    if os.path.exists(npy_file_path):
        # Load the existing data dictionary from the .npy file
        existing_data_dict = np.load(npy_file_path, allow_pickle=True).item()
        # Append the selected columns to the existing data dictionary
        existing_data_dict[sheet_name] = pd.to_numeric(selected_columns, errors='coerce').dropna()
        # Save the updated data dictionary back to the .npy file
        np.save(npy_file_path, existing_data_dict)
    else:
        # Create a new data dictionary and save it as a .npy file
        data_dict[sheet_name] = pd.to_numeric(selected_columns, errors='coerce').dropna()
        np.save(npy_file_path, data_dict)

    return npy_file_path



# Some C1 records antenna current, code commented out included as needed
ifn = r"C:\data\epfl\CRDS\scope trc\bdcageAnt\C1-300W-1mT-00000.trc"
Iant, tarr = read_trc_data(ifn)
Iant *= 8 # Pearson 8A/V

if "C4" in file:
    Vtrig, tarr = read_trc_data(file)
    plt.figure(2)
    plt.plot(tarr*1e3, Vtrig, label = str(power) + "W")
    plt.ylabel('TTL')
    plt.xlabel('time (ms)')
    plt.legend()

In [ ]:
fig = plt.figure()
plt.scatter(tarr_crds, sig_crds)

# signal baseline
start = 200
stop = 240
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_base = np.average(sig_crds[start:stop])

# signal active-glow equilibrium value
start = -70
stop = -20
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_val = np.average(sig_crds[start:stop])

print('%.2e' %(sig_val-sig_base))

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
fig.suptitle('5mTorr/0.67Pa 580W')

ax[0].plot(tarr*1e3, light)
ax[0].set_ylabel('Light')

ax[1].plot(tarr*1e3, Vtrig)
ax[1].set_ylabel('TTL')

tarr_fake = tarr_crds + tarr_crds[-1]
tarr_crds_mod = np.hstack((tarr_crds, tarr_fake)) #+tarr_crds[100]
sig_crds_mod = np.tile(sig_crds , 2) #numpy.roll(sig_crds,-100)

ax[2].plot(tarr_crds_mod*1e3, sig_crds_mod)
ax[2].set_ylabel('crds signal')
ax[2].set_xlabel('time (ms)')